In [33]:
import requests
import datetime
import pandas as pd
import requests
import hopsworks
from pathlib import Path
import json
import re
import os
import warnings
from dotenv import load_dotenv
import openmeteo_requests
import requests_cache
from retry_requests import retry
from xgboost import XGBRegressor

In [ ]:
# Load environment variables from .env file
import os, json
from pathlib import Path
from dotenv import load_dotenv

# Loads .env locally if it exists (in GitHub Actions, it usually doesn't)
load_dotenv()

HW_API_KEY = os.getenv("HOPSWORKS_API_KEY")
HW_PROJECT = os.getenv("HOPSWORKS_PROJECT")

if not HW_API_KEY or not HW_PROJECT:
    raise ValueError("Missing HOPSWORKS_API_KEY or HOPSWORKS_PROJECT environment variables")

# Repo root in GitHub Actions; fallback to current working directory locally
root_dir = Path(os.getenv("GITHUB_WORKSPACE", Path.cwd()))

# Read locations.json
locations_path = root_dir / "locations.json"
if not locations_path.exists():
    raise FileNotFoundError(f"Could not find locations.json at: {locations_path}")

with open(locations_path, "r", encoding="utf-8") as f:
    locations = json.load(f)

# location = locations[0] # Choose location manually when running locally

# city = location["city"]
# latitude = location["latitude"]
# longitude = location["longitude"]
# price_area = location["price_area"]

# # Read location from github actions environment variables if available
# if os.environ.get("CITY"):
#     city = os.environ.get("CITY")
# if os.environ.get("LATITUDE"):
#     latitude = os.environ.get("LATITUDE")
# if os.environ.get("LONGITUDE"):
#     longitude = os.environ.get("LONGITUDE")
# if os.environ.get("PRICE_AREA"):
#     price_area = os.environ.get("PRICE_AREA")

# Connect to Hopsworks project and feature store
project = hopsworks.login(
    project=HW_PROJECT,
    api_key_value=HW_API_KEY,
    host="eu-west.cloud.hopsworks.ai",
)
fs = project.get_feature_store()

2026-01-11 21:41:09,889 INFO: Closing external client and cleaning up certificates.
2026-01-11 21:41:09,892 INFO: Connection closed.
2026-01-11 21:41:09,893 INFO: Initializing external client
2026-01-11 21:41:09,893 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-11 21:41:10,559 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2186


In [ ]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

model_path = root_dir / "electricity_price_model" / "model_electricity_price.json"

retrieved_xgboost_model.load_model(str(model_path))

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",[0.7882746]
,booster,'gbtree'
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import lo

In [36]:
today = datetime.date.today()

In [37]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
batch_data = weather_fg.filter(weather_fg.date >= today).read()
batch_data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.71s) 


,date,temperature_2m_stockholm,precipitation_stockholm,cloud_cover_stockholm,wind_speed_10m_stockholm,temperature_2m_uppsala,precipitation_uppsala,cloud_cover_uppsala,wind_speed_10m_uppsala,temperature_2m_vasteras,...,wind_speed_10m_karlstad,temperature_2m_sundsvall,precipitation_sundsvall,cloud_cover_sundsvall,wind_speed_10m_sundsvall,temperature_2m_malmo,precipitation_malmo,cloud_cover_malmo,wind_speed_10m_malmo,weather_key
0,2026-01-11 18:00:00,-3.2825,0.3,100.0,20.519999,-4.437,0.0,100.0,24.480000,-7.622,...,7.920000,-15.533501,0.0,77.0,10.080000,-2.6955,0.0,0.0,14.759999,se3_set_v1
1,2026-01-11 02:00:00,-3.3325,0.0,100.0,32.399998,-5.737,0.0,100.0,25.199999,-8.872,...,12.959999,-14.483500,0.0,100.0,11.159999,-4.0455,0.0,0.0,32.399998,se3_set_v1
2,2026-01-11 03:00:00,-3.6825,0.0,100.0,31.319998,-5.887,0.0,100.0,24.480000,-9.222,...,12.959999,-13.983500,0.0,95.0,11.520000,-4.1955,0.0,0.0,29.879999,se3_set_v1
3,2026-01-11 06:00:00,-4.3325,0.0,100.0,23.039999,-6.087,0.0,100.0,21.959999,-10.172,...,12.959999,-12.683500,0.0,99.0,8.280000,-3.7955,0.0,0.0,29.519999,se3_set_v1
4,2026-01-11 20:00:00,-3.2325,0.2,100.0,15.480000,-4.037,0.2,100.0,21.240000,-7.372,...,6.840000,-15.683500,0.0,63.0,9.720000,-2.9455,0.0,0.0,14.400000,se3_set_v1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,2026-01-13 16:00:00,-2.8000,0.0,100.0,10.799999,-3.300,0.0,100.0,11.328566,-2.450,...,17.228359,-3.600000,0.0,100.0,5.119727,1.5915,0.0,100.0,17.325874,se3_set_v1
163,2026-01-15 01:00:00,-0.8000,0.1,100.0,19.523983,-1.400,0.1,100.0,19.036650,-0.850,...,19.039200,-0.950000,0.1,100.0,15.858398,3.3415,0.1,100.0,24.136784,se3_set_v1
164,2026-01-16 15:00:00,2.7500,0.4,100.0,19.424160,2.300,0.4,100.0,17.423777,2.000,...,20.052031,0.650000,0.1,100.0,9.236298,4.2915,0.0,100.0,21.816412,se3_set_v1
165,2026-01-16 18:00:00,2.6000,0.3,100.0,20.499462,2.300,0.6,100.0,20.022118,2.000,...,18.307865,1.450000,0.2,100.0,12.961249,3.9915,0.0,81.0,20.302532,se3_set_v1


In [38]:
# Making predictions
feature_cols = [
    # Stockholm
    "temperature_2m_stockholm", "precipitation_stockholm", "cloud_cover_stockholm", "wind_speed_10m_stockholm",
    # Uppsala
    "temperature_2m_uppsala", "precipitation_uppsala", "cloud_cover_uppsala", "wind_speed_10m_uppsala",
    # Västerås
    "temperature_2m_vasteras", "precipitation_vasteras", "cloud_cover_vasteras", "wind_speed_10m_vasteras",
    # Örebro
    "temperature_2m_orebro", "precipitation_orebro", "cloud_cover_orebro", "wind_speed_10m_orebro",
    # Karlstad
    "temperature_2m_karlstad", "precipitation_karlstad", "cloud_cover_karlstad", "wind_speed_10m_karlstad",
    # Sundsvall
    "temperature_2m_sundsvall", "precipitation_sundsvall", "cloud_cover_sundsvall", "wind_speed_10m_sundsvall",
    # Malmö
    "temperature_2m_malmo", "precipitation_malmo", "cloud_cover_malmo", "wind_speed_10m_malmo",
]

batch_data["predicted_sek_per_kwh"] = retrieved_xgboost_model.predict(batch_data[feature_cols])

batch_data

,date,temperature_2m_stockholm,precipitation_stockholm,cloud_cover_stockholm,wind_speed_10m_stockholm,temperature_2m_uppsala,precipitation_uppsala,cloud_cover_uppsala,wind_speed_10m_uppsala,temperature_2m_vasteras,...,temperature_2m_sundsvall,precipitation_sundsvall,cloud_cover_sundsvall,wind_speed_10m_sundsvall,temperature_2m_malmo,precipitation_malmo,cloud_cover_malmo,wind_speed_10m_malmo,weather_key,predicted_sek_per_kwh
0,2026-01-11 18:00:00,-3.2825,0.3,100.0,20.519999,-4.437,0.0,100.0,24.480000,-7.622,...,-15.533501,0.0,77.0,10.080000,-2.6955,0.0,0.0,14.759999,se3_set_v1,2.071169
1,2026-01-11 02:00:00,-3.3325,0.0,100.0,32.399998,-5.737,0.0,100.0,25.199999,-8.872,...,-14.483500,0.0,100.0,11.159999,-4.0455,0.0,0.0,32.399998,se3_set_v1,1.732150
2,2026-01-11 03:00:00,-3.6825,0.0,100.0,31.319998,-5.887,0.0,100.0,24.480000,-9.222,...,-13.983500,0.0,95.0,11.520000,-4.1955,0.0,0.0,29.879999,se3_set_v1,1.282024
3,2026-01-11 06:00:00,-4.3325,0.0,100.0,23.039999,-6.087,0.0,100.0,21.959999,-10.172,...,-12.683500,0.0,99.0,8.280000,-3.7955,0.0,0.0,29.519999,se3_set_v1,0.707003
4,2026-01-11 20:00:00,-3.2325,0.2,100.0,15.480000,-4.037,0.2,100.0,21.240000,-7.372,...,-15.683500,0.0,63.0,9.720000,-2.9455,0.0,0.0,14.400000,se3_set_v1,2.008265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,2026-01-13 16:00:00,-2.8000,0.0,100.0,10.799999,-3.300,0.0,100.0,11.328566,-2.450,...,-3.600000,0.0,100.0,5.119727,1.5915,0.0,100.0,17.325874,se3_set_v1,1.397540
163,2026-01-15 01:00:00,-0.8000,0.1,100.0,19.523983,-1.400,0.1,100.0,19.036650,-0.850,...,-0.950000,0.1,100.0,15.858398,3.3415,0.1,100.0,24.136784,se3_set_v1,0.646018
164,2026-01-16 15:00:00,2.7500,0.4,100.0,19.424160,2.300,0.4,100.0,17.423777,2.000,...,0.650000,0.1,100.0,9.236298,4.2915,0.0,100.0,21.816412,se3_set_v1,0.618670
165,2026-01-16 18:00:00,2.6000,0.3,100.0,20.499462,2.300,0.6,100.0,20.022118,2.000,...,1.450000,0.2,100.0,12.961249,3.9915,0.0,81.0,20.302532,se3_set_v1,0.628356


In [39]:
# Fill in the number of days before the date on which you made the forecast (base_date)
batch_data = batch_data.sort_values(by=['date'])
batch_data['hours_before_forecast_day'] = range(1, len(batch_data)+1)
batch_data["price_area"] = "SE3"
batch_data

,date,temperature_2m_stockholm,precipitation_stockholm,cloud_cover_stockholm,wind_speed_10m_stockholm,temperature_2m_uppsala,precipitation_uppsala,cloud_cover_uppsala,wind_speed_10m_uppsala,temperature_2m_vasteras,...,cloud_cover_sundsvall,wind_speed_10m_sundsvall,temperature_2m_malmo,precipitation_malmo,cloud_cover_malmo,wind_speed_10m_malmo,weather_key,predicted_sek_per_kwh,hours_before_forecast_day,price_area
19,2026-01-11 01:00:00,-3.0825,0.0,100.0,32.039997,-5.737,0.0,100.0,25.919998,-8.272,...,100.0,11.159999,-4.0955,0.0,0.0,30.960001,se3_set_v1,2.221189,1,SE3
1,2026-01-11 02:00:00,-3.3325,0.0,100.0,32.399998,-5.737,0.0,100.0,25.199999,-8.872,...,100.0,11.159999,-4.0455,0.0,0.0,32.399998,se3_set_v1,1.732150,2,SE3
2,2026-01-11 03:00:00,-3.6825,0.0,100.0,31.319998,-5.887,0.0,100.0,24.480000,-9.222,...,95.0,11.520000,-4.1955,0.0,0.0,29.879999,se3_set_v1,1.282024,3,SE3
22,2026-01-11 04:00:00,-3.8325,0.0,100.0,29.519999,-5.887,0.0,99.0,23.759998,-9.622,...,95.0,10.799999,-3.9955,0.0,0.0,29.519999,se3_set_v1,1.226843,4,SE3
9,2026-01-11 05:00:00,-4.5825,0.0,100.0,22.319998,-6.087,0.0,100.0,22.319998,-9.972,...,99.0,9.000000,-3.9955,0.0,0.0,29.519999,se3_set_v1,1.160876,5,SE3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,2026-01-17 19:00:00,1.2000,0.0,100.0,12.320471,0.850,0.0,100.0,12.514855,0.950,...,100.0,10.188700,2.6415,0.0,100.0,11.190442,se3_set_v1,1.418217,163,SE3
59,2026-01-17 20:00:00,1.2000,0.0,100.0,11.866355,0.850,0.0,100.0,11.844492,0.950,...,100.0,9.693296,2.6415,0.0,100.0,11.503113,se3_set_v1,1.517352,164,SE3
137,2026-01-17 21:00:00,1.2500,0.0,100.0,11.503113,0.850,0.0,100.0,11.391313,0.950,...,100.0,9.139322,2.6915,0.0,100.0,11.874544,se3_set_v1,1.533231,165,SE3
120,2026-01-17 22:00:00,1.3000,0.0,100.0,10.985900,0.900,0.0,100.0,10.867290,0.950,...,100.0,8.490582,2.7415,0.0,100.0,12.251904,se3_set_v1,1.514063,166,SE3


In [40]:
# Get or create feature group
monitor_fg = fs.get_or_create_feature_group(
    name='price_predictions',
    description='Electricity price prediction monitoring',
    version=1,
    primary_key=['date','hours_before_forecast_day',"price_area"],
    event_time="date"
)

In [41]:
monitor_fg.insert(batch_data, wait=True)

2026-01-11 21:41:35,777 INFO: Computing insert statistics


(None, None)